# Dataset: Opensus -> System Information Mortality - Brazil. 

Ref: https://opendatasus.saude.gov.br/dataset/sim-2020-2021

"The Mortality Information System (SIM), developed by the Ministry of Health in 1975, is the product of the unification of more than forty Death Certificate models used over the years to collect data on mortality in the country. With its long series time, the SIM is a national heritage, since it has fundamental information so that we can know the aspects related to mortality in Brazil and the causes of illness that led to death. It is also one of the main instruments to support the elaboration of public policies more effective health and social security systems aimed at prevention, promotion and health care."
https://svs.aids.gov.br/daent/cgiae/sim/apresentacao/

## Import packages and load data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
%reload_ext watermark
%watermark -a "Rodrigo S. Pereira" --iversions

Author: Rodrigo S. Pereira

matplotlib: 3.6.0
pandas    : 1.4.1
seaborn   : 0.11.2
numpy     : 1.24.2



In [3]:
# Load the dataset
data = pd.read_parquet('data/DO22OPEN.parquet', engine='pyarrow')

In [4]:
data.head()

,contador,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,...,TPRESGINFO,TPNIVELINV,DTCADINF,MORTEPARTO,DTCONCASO,ALTCAUSA,CAUSABAS_O,TPPOS,TP_ALTERA,CB_ALT
0,1,1,2,14012022,NaN,831.0,316470.0,22051942.0,479.0,1,...,NaN,None,NaN,NaN,NaN,NaN,C229,N,NaN,None
1,2,1,2,15012022,1350.0,NaN,NaN,17042001.0,420.0,1,...,NaN,None,NaN,NaN,NaN,NaN,Y299,N,NaN,None
2,3,1,2,19012022,630.0,835.0,352700.0,24061930.0,491.0,1,...,NaN,None,NaN,NaN,NaN,NaN,B342,S,NaN,None
3,4,1,2,23012022,315.0,835.0,350030.0,6071931.0,490.0,2,...,NaN,None,NaN,NaN,NaN,NaN,B342,N,NaN,None
4,5,1,2,21012022,2151.0,825.0,250110.0,17121920.0,501.0,1,...,NaN,None,NaN,NaN,NaN,NaN,B342,S,NaN,None


In [5]:
print(f'The data Shape n_rows and n_cols are: {data.shape}')

The data Shape n_rows and n_cols are: (1031744, 86)


## Data Preparations

In [6]:
# Verify the columns:
data.columns

Index(['contador', 'ORIGEM', 'TIPOBITO', 'DTOBITO', 'HORAOBITO', 'NATURAL',
       'CODMUNNATU', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC',
       'ESC2010', 'SERIESCFAL', 'OCUP', 'CODMUNRES', 'LOCOCOR', 'CODESTAB',
       'CODMUNOCOR', 'IDADEMAE', 'ESCMAE', 'ESCMAE2010', 'SERIESCMAE',
       'OCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'SEMAGESTAC',
       'GESTACAO', 'PARTO', 'OBITOPARTO', 'PESO', 'TPMORTEOCO', 'OBITOGRAV',
       'OBITOPUERP', 'ASSISTMED', 'EXAME', 'CIRURGIA', 'NECROPSIA', 'LINHAA',
       'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'CAUSABAS', 'CB_PRE',
       'COMUNSVOIM', 'DTATESTADO', 'CIRCOBITO', 'ACIDTRAB', 'FONTE',
       'NUMEROLOTE', 'DTINVESTIG', 'DTCADASTRO', 'ATESTANTE', 'STCODIFICA',
       'CODIFICADO', 'VERSAOSIST', 'VERSAOSCB', 'FONTEINV', 'DTRECEBIM',
       'ATESTADO', 'DTRECORIGA', 'OPOR_DO', 'CAUSAMAT', 'ESCMAEAGR1',
       'ESCFALAGR1', 'STDOEPIDEM', 'STDONOVA', 'DIFDATA', 'NUDIASOBCO',
       'DTCADINV', 'TPOBITOCOR', 'DTCONINV',

Notes: 
    - The data set has a total of **1031744 records** and **86 variables**; 
    - From the variables present in the original dataset, we can exclude `contador`, `ORIGEM`;
   
Select the variable for the fisrt analysis:   
    - `TIPOBITO`: Fetal/Nao Fetal;  
    - `DTOBITO`: Data do óbito formato ddmmaaa.  
    - `DTNASC`: Data do nascimento do falecido, formato ddmmaaaa.   
    - `IDADE`: Idade do falecido em minutos, horas, dias, meses ou anos.  
.    - `SEXO`:.  
    - `RACACOR`: raça e cor.  
    - `ESTCIV`: estado civil.  
    - `ESC2010`: nível da escolaridade.  
    - `ESC`: Escolaridade por tempo.  
    - `OCUP`: Tipo de trabalho que o falecido desenvolveu na maior parte de sua vida
            produtiva.  
    - `LOCOCOR`: Local de ocorrencia do óbito.  
    - `CODMUNOCOR`: código do municipio onde ocorreu o óbito.  
    - `OBITOGRAV`: Óbito na gravidez.  
    - `OBITOPUERP`: Óbito no puerpério.  
    - `LINHAA`:.  
    - `LINHAB`:.   
    - `LINHAC`:.  
    - `LINHAD`:.  
    - `LINHAII`:.  
    - `CAUSABAS`: Causa básica da DO.   
    - `CIRCOBITO`: Tipo de morte violenta ou circunstâncias em que se deu a morte não natural.  
    - `ACIDTRAB`: Indica se o evento que desencadeou o óbito está relacionado ao processo de
                trabalho.  
    - `ASSISTMED`: assistencia médica antes da ocorrencia do óbito.  
    
    

In [7]:
# Select the variables for this first data exploratory: 
select_var = ['TIPOBITO', 'DTOBITO', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'ESTCIV','ESC2010',
             'ESC', 'OCUP', 'LOCOCOR', 'CODMUNOCOR', 'OBITOGRAV', 'OBITOPUERP', 'LINHAA', 'LINHAB',
             'LINHAC', 'LINHAD', 'LINHAII', 'CAUSABAS', 'CIRCOBITO', 'ACIDTRAB', 'ASSISTMED']

**Put here some explanation, why this select_var set** #########

In [8]:
df = data[select_var]
df.shape

(1031744, 23)

In [9]:
# Function for count and return new dataframe with the relation about null values
def count_df_null(data):
    df_l = pd.DataFrame(data.isnull().sum().sort_values(), columns=['Total null'])
    df_l['%'] = df_l['Total null']/data.shape[0]*100
    return df_l

In [10]:
# Verify the value null in all dataframe and that % of total observations
count_df_null(df)

,Total null,%
TIPOBITO,0,0.000000
CAUSABAS,0,0.000000
LOCOCOR,0,0.000000
SEXO,0,0.000000
CODMUNOCOR,0,0.000000
DTOBITO,0,0.000000
IDADE,17718,1.717286
DTNASC,19701,1.909485
LINHAA,32587,3.158439
RACACOR,38227,3.705086


* We consider trunced variable with no more with 20% of lost data. 

In [11]:
select_var = count_df_null(df)[count_df_null(df)['%'] < 20].index.values
select_var

array(['TIPOBITO', 'CAUSABAS', 'LOCOCOR', 'SEXO', 'CODMUNOCOR', 'DTOBITO',
       'IDADE', 'DTNASC', 'LINHAA', 'RACACOR', 'ESTCIV', 'ESC', 'ESC2010',
       'OCUP'], dtype=object)

In [12]:
# Filtering the data by select_var and atribute to new df2: 
df2 = df[select_var]
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031744 entries, 0 to 1031743
Data columns (total 14 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   TIPOBITO    1031744 non-null  int64  
 1   CAUSABAS    1031744 non-null  object 
 2   LOCOCOR     1031744 non-null  int64  
 3   SEXO        1031744 non-null  int64  
 4   CODMUNOCOR  1031744 non-null  int64  
 5   DTOBITO     1031744 non-null  int64  
 6   IDADE       1014026 non-null  float64
 7   DTNASC      1012043 non-null  float64
 8   LINHAA      999157 non-null   object 
 9   RACACOR     993517 non-null   float64
 10  ESTCIV      967001 non-null   float64
 11  ESC         949117 non-null   float64
 12  ESC2010     943687 non-null   float64
 13  OCUP        869722 non-null   float64
dtypes: float64(7), int64(5), object(2)
memory usage: 110.2+ MB


Select process for the representative variable, now we need to work on the change some data for put in better visualization and a post processing of the exploratory data. 

Relation of the select variable and how is the process for apply: 
* `TIPOBITO`: Change the number 1 to fetal and 2 to not fetal.

In [13]:
# Ajustment of var TIPOBITO: 
df2.TIPOBITO = df2.TIPOBITO.replace(2, 'Not_Fetal')
df2.TIPOBITO = df2.TIPOBITO.replace(1, 'Fetal')

In [14]:
df2.shape
print(f'The new data df2 Shape n_rows and n_cols are: {df2.shape}')

The new data df2 Shape n_rows and n_cols are: (1031744, 14)


In [15]:
df2.head(3)

,TIPOBITO,CAUSABAS,LOCOCOR,SEXO,CODMUNOCOR,DTOBITO,IDADE,DTNASC,LINHAA,RACACOR,ESTCIV,ESC,ESC2010,OCUP
0,Not_Fetal,C229,3,1,352050,14012022,479.0,22051942.0,*J180,1.0,NaN,9.0,9.0,999993.0
1,Not_Fetal,Y299,1,1,355620,15012022,420.0,17042001.0,*S069,1.0,1.0,9.0,9.0,NaN
2,Not_Fetal,B342,1,1,350950,19012022,491.0,24061930.0,*J960,1.0,2.0,3.0,1.0,999993.0


In [16]:
# Verify the df2 na values: 
count_df_null(df2)

,Total null,%
TIPOBITO,0,0.000000
CAUSABAS,0,0.000000
LOCOCOR,0,0.000000
SEXO,0,0.000000
CODMUNOCOR,0,0.000000
DTOBITO,0,0.000000
IDADE,17718,1.717286
DTNASC,19701,1.909485
LINHAA,32587,3.158439
RACACOR,38227,3.705086


* **The observations with contain nan values for this variables above are substituted for "not_informed"**

In [17]:
# Treatment of nan values: 
df2.fillna(value='not_informed', inplace=True)

In [18]:
df2.head(3)

,TIPOBITO,CAUSABAS,LOCOCOR,SEXO,CODMUNOCOR,DTOBITO,IDADE,DTNASC,LINHAA,RACACOR,ESTCIV,ESC,ESC2010,OCUP
0,Not_Fetal,C229,3,1,352050,14012022,479.0,22051942.0,*J180,1.0,not_informed,9.0,9.0,999993.0
1,Not_Fetal,Y299,1,1,355620,15012022,420.0,17042001.0,*S069,1.0,1.0,9.0,9.0,not_informed
2,Not_Fetal,B342,1,1,350950,19012022,491.0,24061930.0,*J960,1.0,2.0,3.0,1.0,999993.0



**Summary transformation about Variables:** 

    - *SEXO: {0:"ignored", 1:"masculine", 2:"feminine"} 
    - *LOCOCOR: Place of occurrence of death; We need to change the class number to word for help the future representations. {1:"hospital", 2:"others hospitals", 3:"health", 4:"public highway", 5:"others", 6:"indian village", 9:"ignored"};
    
    - DTOBITO: Death date ( dd mm aaaa) - Need to change this col for a type datetime;
    
    - *IDADE: 1ºX-2ºXX - for the 1ºX={1:minute; 2:hour; 3:month; 4:year; 5:secular}
                                2ºXX={unit of measurement respective the 1ºX};
    
    - DTNASC: Born date (dd mm aaaa) - Need to change to a type datetime;
    
    - *RACACOR: ethnic identity - {1: white, 2: black, 3:yellow, 4:Brown, 5:Indigenous}
    * Observation: It is important to note that these terms are not widely accepted in English, as they are often considered to be reductive and potentially offensive. Instead of using these terms, it is more appropriate to use terms that accurately and respectfully describe a person's racial or ethnic identity. For example, "White" can be described as "Caucasian" or "European American", "Black" can be described as "African American", "Yellow" is not a commonly used term, "Brown" can be described as "Latino" or "Hispanic", and "Indigenous" can be described as "Native American" or "Aboriginal".
    
     - *ESTCIV: Marital status of the deceased as reported by relatives. {1:Single; 2:Married; 3:Widowed; 4:separated/divorced; 5:Common-law marriage; 9:Unknown)
    
    - *ESC: Years of schooling. {1:"None", 2:"1 to 3 years", 3:"4 to 7 years", 4:"8 to 11 years", 5:"12 or more years", 9:"Unknown"};
    
    - *ESC2010: Level of the last grade completed by the deceased. {0:"No education", 1:"Primary education (1st to 4th grade)", 2:"Secondary education (5th to 8th grade)", 3:"High school (former 2nd degree)", 4:"Incomplete higher education"; 5:"Completed higher education", 9:"Unknown"};
    
    
   

In [19]:
# Create a relation variable for the LOCOCOR, IDADE, RACACOR, ESTCIV, ESC, ESC2010 and SEXO
dic_LOCOCOR = {1:"hospital", 2:"others hospitals", 3:"health", 4:"public highway", 
               5:"others", 6:"indian village", 9:"ignored"}
dic_SEXO = {0:"ignored", 1:"masculine", 2:"feminine"}
dic_IDADE = {1:"minute", 2:"hour", 3:"month", 4:"year", 5:"secular"}
dic_RACACOR = {1: "white", 2:"black", 3:"yellow", 4:"Brown", 5:"Indigenous"}
dic_ESTCIV = {1:"Single", 2:"Married", 3:"Widowed", 4:"separated/divorced", 5:"Common-law marriage", 9:"Unknown"}
dic_ESC = {1:"None", 2:"1 to 3 years", 3:"4 to 7 years", 4:"8 to 11 years", 5:"12 or more years", 9:"Unknown"}
dic_ESC2010 = {0:"No education", 1:"Primary education (1st to 4th grade)", 2:"Secondary education (5th to 8th grade)",
               3:"High school (former 2nd degree)", 4:"Incomplete higher education", 5:"Completed higher education", 9:"Unknown"}
columns_to_update = {'LOCOCOR': dic_LOCOCOR,
                     'RACACOR': dic_RACACOR,
                     'ESTCIV': dic_ESTCIV,
                     'ESC': dic_ESC,
                     'ESC2010': dic_ESC2010,
                     'SEXO': dic_SEXO}

In [20]:
# Update the columns_to_update
def update_column_values(df, column, dic_name):
    df[column] = df[column].apply(lambda x: x if x == 'not_informed' else dic_name[x])

for column, dic_name in columns_to_update.items():
    update_column_values(df2, column, dic_name)

In [21]:
df2.head(3)

,TIPOBITO,CAUSABAS,LOCOCOR,SEXO,CODMUNOCOR,DTOBITO,IDADE,DTNASC,LINHAA,RACACOR,ESTCIV,ESC,ESC2010,OCUP
0,Not_Fetal,C229,health,masculine,352050,14012022,479.0,22051942.0,*J180,white,not_informed,Unknown,Unknown,999993.0
1,Not_Fetal,Y299,hospital,masculine,355620,15012022,420.0,17042001.0,*S069,white,Single,Unknown,Unknown,not_informed
2,Not_Fetal,B342,hospital,masculine,350950,19012022,491.0,24061930.0,*J960,white,Married,4 to 7 years,Primary education (1st to 4th grade),999993.0


* Obs: The most values unique demonstrate with have some patterns with just two digits, like example "50" or "16", and this don't represent the valid number for IDADE based on the subtitle of dictionary on document.

* For this analisys we assume just the frequency time about completed years, for values with represent less than year we assum new value like "less_year". And the values with contain two digits is represent by "not_inform". 

In [24]:
#Function for select the 2 digits of variable IDADE and change for "not_inform"
def select_digits(digit):
    digit_str = str(int(digit))
    if len(digit_str) < 3:
        return 0
    if digit_str[0] in '123':
        return 9
    return int(digit)

In [25]:
# Apply the select_digits:
df2.IDADE = df2.IDADE.apply(lambda x: select_digits(x) if x != 'not_informed' else 0)

# Create a new column by represent the frequency time: 
dic_IDADE.update({0:'not_informed', 9:'less_year'})
df2.insert(7, column='Freq_time', value=df2.IDADE.apply(lambda x: dic_IDADE[int(list(str(x))[0])]))

# Modify the var IDADE, exclude the first digit when is 4 and change for 1 when is 5: 
def modify_age(cod):
    condition = (str(cod).startswith('5') and len(str(cod)) == 3) or (str(cod).startswith('4') and len(str(cod)) == 3)
    if condition:
        return int(str(cod)[1:])
    else:
        return cod
    
    
df2.IDADE = df2.IDADE.apply(lambda x: modify_age(x))

In [36]:
df2.head(3)

,TIPOBITO,CAUSABAS,LOCOCOR,SEXO,CODMUNOCOR,DTOBITO,IDADE,Freq_time,DTNASC,LINHAA,RACACOR,ESTCIV,ESC,ESC2010,OCUP
0,Not_Fetal,C229,health,masculine,352050,14012022,79,year,22051942.0,*J180,white,not_informed,Unknown,Unknown,999993.0
1,Not_Fetal,Y299,hospital,masculine,355620,15012022,20,year,17042001.0,*S069,white,Single,Unknown,Unknown,not_informed
2,Not_Fetal,B342,hospital,masculine,350950,19012022,91,year,24061930.0,*J960,white,Married,4 to 7 years,Primary education (1st to 4th grade),999993.0


In [39]:
df2.LINHAA.unique()

array(['*J180', '*S069', '*J960', ..., '*X930*T794', '*I509*R578',
       '*A418*J22X'], dtype=object)

More Summary Variable Transformations: 

- LINHAA: Cod CD10 of the terminal cause of Death); (Org this)
- CAUSABAS: Cod CID10 ( Basic cause of DO) (Org this);
- CODMUNOCOR : Cod. City Brazil ( Org this);
- OCUP: Cod. of type work Brazil CBO(Classificação Brasileira de Ocupação) (Org this);